In [0]:
import numpy as np
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.python.keras.layers import LSTM, GRU, TimeDistributed, Input, Conv1D, Dropout, Reshape, Dense, Softmax, BatchNormalization, MaxPooling1D, AveragePooling1D, Flatten
from tensorflow.python.keras.models import Sequential, Model

import pandas as pd
import subprocess

from sklearn.model_selection import train_test_split

import random

# Load data

In [2]:
from google.colab import drive
drive.mount('/drive/', force_remount=True)
DRIVE_PATH = "/drive/My Drive/"

Mounted at /drive/


In [3]:
# Change working directory
import os
os.chdir("/drive/My Drive/dataCollab")
!pwd

/drive/My Drive/dataCollab


In [0]:
for k in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
  for i in range (0, 15):
    max_movements = 4 #random.randint(3,5)
    cmd = "python generator.py " + k + " " + str(i) + " " + str(max_movements)
    process = subprocess.Popen(cmd, shell=True,
                             stdout=subprocess.PIPE, 
                             stderr=subprocess.PIPE)
    out, err = process.communicate()
    #print(out.decode("utf-8"))
    print("Generated ("+ k +") " + str(i), end='\n', flush=True)

print("")

In [4]:
X_files = os.listdir('exercises')
Y_files = os.listdir('results')
X_len = len(X_files)
Y_len = len(Y_files)

# 4000 samples more or less
MAX_SENSORS = 12
MAX_MOVEMENTS = 8
MAX_SAMPLES = 4000

#X = np.zeros((X_len,MAX_SAMPLES,MAX_SENSORS))
#Y = np.zeros((Y_len,MAX_SAMPLES,MAX_MOVEMENTS))

'''loaded = 0
X_mats = []
for file in X_files:
  try:
    mat = pd.read_csv('exercises/' + file).fillna(0).values
    
    #if mat.shape[0] > 4000:
    X_mats.append(mat)
    loaded += 1
  except:
    pass
print('Loaded [X] ' + str(loaded) + ' from ' + str(X_len))

loaded = 0
Y_mats = []
for file in Y_files:
  try:
    mat = pd.read_csv('results/' + file).fillna(0).values
    
    #if mat.shape[0] > 4000:
    Y_mats.append(mat)
    loaded += 1
  except Exception as e:
    print(e)

print('Loaded [Y] ' + str(loaded) + ' from ' + str(Y_len))'''

X_mats = []
Y_mats = []
loaded = 0
for i in range(0, len(X_files)):
  num_seq = int(os.path.splitext(X_files[i])[0].split('-')[-1])
  if num_seq != 4:
    continue
  
  try:
    mat = pd.read_csv('exercises/' + X_files[i]).fillna(0).values
    
    if mat.shape[1] != MAX_SENSORS:
      continue
    
    X_mats.append(mat)
    loaded += 1
  except Exception as e:
    print(e)
    continue
    
  try:
    mat = pd.read_csv('results/' + Y_files[i]).fillna(0).values
    
    Y_mats.append(mat)
    loaded += 1
  except Exception as e:
    print(e)

print(str(len(X_mats)) + " " + str(len(Y_mats)))
    
X_max = max([a.shape[0] for a in X_mats])
for i in range(0, len(X_mats)):
  if X_mats[i].shape[0] < X_max:
    rem = X_max - X_mats[i].shape[0]
    X_mats[i] = np.concatenate((X_mats[i], np.zeros((rem, MAX_SENSORS))), axis=0)

X = np.stack(X_mats)
print(X.shape)

for i in range(0, len(Y_mats)):
  if Y_mats[i].shape[0] < X_max:
    rem = X_max - Y_mats[i].shape[0]
    Y_mats[i] = np.concatenate((Y_mats[i], np.zeros((rem, MAX_MOVEMENTS))), axis=0)  # Poner 1

Y = np.stack(Y_mats)
print(Y.shape)

Error tokenizing data. C error: Expected 12 fields in line 7246, saw 18

Error tokenizing data. C error: Expected 12 fields in line 15005, saw 21

114 114
(114, 46008, 12)
(114, 46008, 8)


In [5]:
# 4000 samples more or less
X = X[:, 0:46000, :]
Y = Y[:, 0:46000, :]
print(X.shape)
print(Y.shape)

(114, 46000, 12)
(114, 46000, 8)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=69)
print("x_train : " + str(x_train.shape) + " x_test : " + str(x_test.shape) + 
      " y_train : " + str(y_train.shape) + " y_test : " + str(y_test.shape))

x_train : (79, 46000, 12) x_test : (35, 46000, 12) y_train : (79, 46000, 8) y_test : (35, 46000, 8)


# Net

In [0]:
def sensorLayer(x, filters, kernels, sizes_LSTM, denses, activation='relu', value_dropout=0.45):
  length = x.shape[0].value
  length_time = x.shape[1].value
  
  # Convolutions
  for i in range(0, len(filters)):
    x = Conv1D(filters[i], kernel_size=kernels[i], activation=activation, padding="same")(x)
    x = Dropout(value_dropout)(x)
    x = MaxPooling1D(pool_size=3)(x)
    #x = AveragePooling1D(2)(x)
    
  # LSTM
  for i in range(0, len(sizes_LSTM)):
    #x = LSTM(sizes_LSTM[i], return_sequences=True)(x)
    x = GRU(sizes_LSTM[i], return_sequences=True)(x)
  
  # Flatten for classsify
  x = Flatten()(x)
  #x = Reshape([-1, length_time, -1])(x)
  #x = TimeDistributed(Flatten())(x)
  
  for i in range(0, len(denses)-1):
    x = Dense(denses[i],activation='relu')(x)
  
  if len(denses) != 0:
     x = Dense(denses[-1],activation=None)(x)
  
  x = Dropout(value_dropout)(x)
  
  x = Softmax()(x)
  
  return x
  
  

In [0]:
N_SAMPLES = 4000
N_SENSORS = 12
N_CLASSES = 8
FILTERS = [30, 30, 15]
KERNELS = [25, 50, 100]
HIDDEN_LSTM = []
DENSE = [256, N_CLASSES]

#[BATCH, steps, feature]

# Generate all inputs
inputs = Input(shape=(N_SAMPLES, N_SENSORS), name='inputs')

normalized = BatchNormalization(axis=2)(inputs)

prediction = sensorLayer(normalized, FILTERS, KERNELS, HIDDEN_LSTM, DENSE)

model = Model(inputs=inputs, outputs=prediction)


In [44]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 4000, 12)          0         
_________________________________________________________________
batch_normalization_v1_3 (Ba (None, 4000, 12)          48        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 4000, 30)          9030      
_________________________________________________________________
dropout_12 (Dropout)         (None, 4000, 30)          0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 1333, 30)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1333, 30)          45030     
_________________________________________________________________
dropout_13 (Dropout)         (None, 1333, 30)          0         
__________

In [0]:
OPTIMIZER = "adam"  # SGD
LOSS = "categorical_crossentropy" # categorical_crossentropy

model.compile(optimizer=OPTIMIZER,
              loss=LOSS, #categorical_crossentropy
              metrics=['accuracy'])

In [41]:
batch_size = 8
n_epochs = 10
# N_SAMPLES

for i in range(0, n_epochs):
  for b in range(0, x_train.shape[0] // batch_size):
    #ind = np.random.permutation(x_train.shape[0])
    ind = np.arange(0, x_train.shape[0], dtype=int)
    indb = ind[b * batch_size: (b+1) * batch_size]
    
    X_2 = x_train[indb]
    Y_2 = y_train[indb]
    
    X_batch_list = []
    Y_batch_list = []
    for l in range(0, X_2.shape[0]):
      for k in range(0, X_2.shape[1] // N_SAMPLES):
        X_batch_list.append(X_2[l, k*N_SAMPLES:(k+1)*N_SAMPLES, :])
        Y_batch_list.append(Y_2[l, k*N_SAMPLES:(k+1)*N_SAMPLES, :])
    
    X_2 = np.stack(X_batch_list)
    Y_2 = np.stack(Y_batch_list)
    
    model.fit(X_2, Y_2, epochs=1, batch_size=1, verbose=1, shuffle=False)
    #model.reset_states()
  print("Epoch => " + str(i))

ValueError: ignored

In [42]:
batch_size = 4
n_epochs = 2

SAMPLES_PAD = N_SAMPLES // 2

np.random.shuffle(x_train)

for i in range(0, n_epochs+1):
  for b in range(0, x_train.shape[0] // batch_size):
    #ind = np.random.permutation(x_train.shape[0])
    ind = np.arange(0, x_train.shape[0], dtype=int)
    indb = ind[b * batch_size: (b+1) * batch_size]
    
    X_2 = x_train[indb]
    Y_2 = y_train[indb]
    
    X_batch_list = []
    Y_batch_list = []
    for l in range(0, X_2.shape[0]):
      #index = SAMPLES_PAD + 1
      index = np.random.randint(SAMPLES_PAD + 1, 40000-SAMPLES_PAD)
      count = 0
      while index <= X_2[l, :, :].shape[0] - SAMPLES_PAD - 1:
        X_batch_list.append(X_2[l, index-SAMPLES_PAD:index+SAMPLES_PAD, :])
        Y_batch_list.append(Y_2[l, index])
        
        index += 20
        count += 1
        
        if count == 500:
          
          rInd = np.random.randint(0, X_2.shape[0])
          
          X_3 = np.stack(X_batch_list)
          Y_3 = np.stack(Y_batch_list)
          
          #X_3 = X_3[rInd, :, :]
          #Y_3 = Y_3[rInd, :]
          
          model.fit(X_3, Y_3, epochs=1, batch_size=1, verbose=1, shuffle=False)
          
          X_batch_list = []
          Y_batch_list = []
          count = 0
    
    #X_2 = np.stack(X_batch_list)
    #Y_2 = np.stack(Y_batch_list)
    
    #model.fit(X_2, Y_2, epochs=1, batch_size=1, verbose=1, shuffle=False)
    #model.reset_states()
  print("Epoch => " + str(i))
  
  # Save model
  model_json = model.to_json()
  with open("model_LSTM.json", "w") as json_file:
    json_file.write(model_json)

  model.save_weights("model_LSTM.h5")
  print("Saved model to disk")

241/984 [======>.......................] - ETA: 3:40 - loss: 0.0000e+00 - acc: 0.2033

KeyboardInterrupt: ignored

In [0]:
#Test the best model got by far
#model.set_weights(best_model) 

X_batch_list = []
Y_batch_list = []
for l in range(0, X_2.shape[0]):
  for k in range(0, X_2.shape[1] // N_SAMPLES):
    X_batch_list.append(X_2[l, k*N_SAMPLES:(k+1)*N_SAMPLES, :])
    Y_batch_list.append(Y_2[l, k*N_SAMPLES:(k+1)*N_SAMPLES, :])

X_2_test = np.stack(X_batch_list)
Y_2_test = np.stack(Y_batch_list)

score = model.evaluate(X_2_test,Y_2_test)

print('error rate: %f %%' % ( (1-score[1]) * 100))

184/184 [==============================] - 0s 3ms/sample - loss: 0.8470 - acc: 0.2139
error rate: 78.613587 %


In [30]:
X_batch_list = []
Y_batch_list = []

total_score = 0
total_n = 0

for l in range(0, x_train.shape[0]):
  index = SAMPLES_PAD + 1
  count = 0
  while index <= x_train[l, :, :].shape[0] - SAMPLES_PAD - 1:
    X_batch_list.append(x_train[l, index-SAMPLES_PAD:index+SAMPLES_PAD, :])
    Y_batch_list.append(y_train[l, index])
    
    index += 5

    count += 1

    if count == 5000:
      X_2_test = np.stack(X_batch_list)
      Y_2_test = np.stack(Y_batch_list)
      
      score = model.evaluate(X_2_test,Y_2_test)
      total_score += score[1]
      total_n += 1
      
      X_batch_list = []
      Y_batch_list = []
      count = 0

#total_score /= float(total_n)

8400/8400 [==============================] - 5s 573us/sample - loss: 9.5960 - acc: 0.4046


In [31]:
total_score /= float(total_n)
print('error rate: %f %%' % ( (1-total_score) * 100))

error rate: 50.737643 %


In [37]:
# Save model
model_json = model.to_json()
with open("model_LSTM.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_LSTM.h5")
print("Saved model to disk")

Saved model to disk
